In [ ]:
import sys
import os
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
os.environ["MX_DEX_ENV"] = "shadowfork4"
from context import Context

context = Context()

In [ ]:
from multiversx_sdk import Address, ApiNetworkProvider
from utils.utils_tx import endpoint_call
import config

# Needed for ShadowFork
api_network_provider = ApiNetworkProvider('https://api.multiversx.com') if "shadowfork" in config.DEFAULT_PROXY else context.network_provider.api

print(f'Deployer account: {context.deployer_account.address.to_bech32()}')
contracts = api_network_provider.do_get_generic(f'accounts/{context.deployer_account.address.to_bech32()}/contracts?size={500}')

print(f'Total number of contracts: {len(contracts)}')

for contract in contracts:
        contract_address = contract['address']
        contract_details = context.network_provider.api.do_get_generic(f'accounts/{contract_address}?fields=developerReward')
        developer_reward = int(contract_details['developerReward'])
        print(f'Contract: {contract_address} -> developer reward: {developer_reward}')
        if developer_reward > 0:
            tx_hash = endpoint_call(context.network_provider.proxy, 20000000, context.deployer_account, Address.new_from_bech32(contract_address), "ClaimDeveloperRewards", [])

In [ ]:
from contracts.router_contract import RouterContract
from utils.contract_data_fetchers import RouterContractDataFetcher
from utils.utils_chain import WrapperAddress

router_contract: RouterContract = context.get_contracts(config.ROUTER_V2)[0]
context.deployer_account.sync_nonce(context.network_provider.proxy)

# Find pairs with developer rewards
router_data_fetcher = RouterContractDataFetcher(Address.new_from_bech32(router_contract.address), context.network_provider.proxy.url)
registered_pairs = router_data_fetcher.get_data("getAllPairsManagedAddresses")
print(f'Found {len(registered_pairs)} pairs')

count = 0
sum = 0
selected_pairs = []
for pair_address in registered_pairs:
    address = WrapperAddress.from_hex(pair_address)
    min_threshold = 0.1 * 10**18
    account_on_chain = context.network_provider.proxy.get_account(address)
    if account_on_chain.contract_developer_reward > min_threshold:
        count += 1
        sum += account_on_chain.contract_developer_reward
        selected_pairs.append(pair_address)

print(f"Found {count} pairs with developer rewards summing: {sum} (min threshold: {min_threshold})")
assert count > 0, "Didn't find any pairs with developer rewards above the threshold."

# Claim developer rewards
chunk_size = 20
b32_pairs = [WrapperAddress.from_hex(pair_address).bech32() for pair_address in selected_pairs]
chunks = [b32_pairs[i:i + chunk_size] for i in range(0, len(b32_pairs), chunk_size)]
for chunk in chunks:
    router_contract.claim_developer_rewards_pairs(context.deployer_account, context.network_provider.proxy, chunk)